In [66]:
import glob
import argparse
import os
import subprocess
import datetime


In [116]:
def get_subjects():
    global sub_dir
    global output_dir
    global input_dir
    global derivatives_dir
    global subjects
    subjects = []
    # Get data from input
    #input_dir = input("Enter directory path of your subjects: ")
    #output_dir = input("Enter directory path for your output: ")
    input_dir = '/projects/niblab/bids_projects/Experiments/Bevel/derivatives'
    output_dir = '/projects/niblab/bids_projects/Experiments/Bevel'
    derivatives_dir = os.path.join(output_dir, 'derivatives')
    sub_dir=glob.glob(os.path.join(input_dir, 'sub*'))
    output_files()
    for path in sub_dir:
        sub = path.split("/")[-1]
        #if 'sub' in file:
        subjects.append(sub)
    for sub in sorted(subjects):
        print(">>>>>>-----------> ", sub)
        fd_check(sub)

In [117]:
def set_paths(sub):
    global func_output_path
    global anat_output_path
    global func_input_path
    global motion_assessment_path
    out_dir = os.path.join(derivatives_dir, sub)
    anat_output_path=os.path.join(out_dir, 'anat')
    func_output_path=os.path.join(out_dir,'func')
    motion_assessment_path=os.path.join(out_dir,'func','motion_assessment')
    #print("> FUNC OUTPUT: ", func_output_path)
    #print("> MOTION OUTPUT: ", motion_assessment_path)

In [118]:

def output_files():
    global datestamp
    global outhtml
    global out_bad_bold_list
    global outfile
    datestamp=datetime.datetime.now().strftime("%Y-%m-%d-%H_%M_%S")
    out_bad_bold_list = os.path.join(derivatives_dir,'TEST_BAD_%s.txt'%(datestamp))
    outhtml = os.path.join(derivatives_dir,'bold_motion_QA_%s.html'%(datestamp))
    outfile = open(outhtml, 'a')

    

In [119]:
def fd_check(sub):
    try:
        print(">>>>>>>>-----------------> STARTING MOTION CORRECTION ON ", sub)
        set_paths(sub)
        for nifti in glob.glob(os.path.join(func_output_path, '*brain.nii.gz')):
            filename=nifti.split('.')[0]
            file = filename.split("/")[-1]
            if "training" in file:
                run_id = "tr_"+file.split("_")[3]
            elif "rl" in file:
                run_id = "rl_"+file.split("_")[3]
            else:
                run_id = "resting"
        # set comparison param
            print(file)
            print("RUN ID >>>>>>>>>--------> ", run_id)
            nvols_cmd="fslnvols " + nifti
            volume = subprocess.check_output(nvols_cmd, shell=True, encoding="utf-8")
            volume = volume.strip()
            comparator = int(volume) *.25
            ## RUN 'fsl_motion_outliers' TO RETRIEVE MOTION CORRECTION ANALYSIS
            outlier_cmd = "fsl_motion_outliers -i %s  -o %s/%s_confound.txt  --fd --thresh=0.9 -p %s/%s_fd_plot -v > %s/%s_outlier_output.txt"%(filename, motion_assessment_path, file, motion_assessment_path,run_id, motion_assessment_path, file)
            print(">>>>>>>>>-------->  RUNNING FSL MOTION OUTLIERS ")
            os.system(outlier_cmd)
        ## EXAMINE OUTLIER FILE AND GRAB RELEVANT DATA 
            outlier_file="%s/%s_outlier_output.txt"%(motion_assessment_path, file)
            with open(outlier_file, 'r') as f:
                lines=f.readlines()
                statsA = lines[1].strip("\n") #maskmean
                statsB = lines[3].strip("\n") #metric range
                statsC = lines[4].strip("\n") #outliers found
                if int(statsC.split(" ")[1])  > 0:
                    statsD = lines[6].strip("\n") #spikes found
                else:
                    statsD = "\n"
            f.close()
        ## GRAB MOTION CORRECTION PLOT AND WRITE PLOT & INFO TO HTML
            plotz=os.path.join(motion_assessment_path, run_id+'_fd_plot.png')
            FILEINFO="""<p><font size=7> <b>{CURR_FILENAME} </b></font><br>"""
            CURR_FILEINFO = FILEINFO.format(CURR_FILENAME=file)
            outfile.write(CURR_FILEINFO)
            INFO="""<p><font size=6>{A} <br><b>{B}<b><br>{C}<br><b>{D}</b><br><br>"""
            CURR_INFO= INFO.format(A=statsA, B=statsB, C=statsC, D=statsD)
            outfile.write(CURR_INFO)
            PLOT="""<IMG SRC=\"{PLOTPATH}\" WIDTH=100%><br><br>"""
            CURR_PLOT = PLOT.format(PLOTPATH=plotz)
            outfile.write(CURR_PLOT)
            print(">>>>>>>>>--------> COPYING OUTPUT TO HTML")
            print(">>>>>>>>>--------> ADDING PLOT TO HTML")
        ## ADD FILE FOR GOOD SUBJECT 
        # --sometimes you have a great subject who didn't move
            if os.path.isfile("%s/%s_confound.txt"%(motion_assessment_path, file))==False:
                os.system("touch %s/%s_confound.txt"%(motion_assessment_path, file))
        ## CHECK FOR BAD SUBJECTS: ABOVE OUR THRESHOLD
        # how many columns are there = how many 'bad' points
            check = subprocess.check_output("grep -o 1 %s/%s_confound.txt | wc -l"%(motion_assessment_path, file), shell=True)
            num_scrub = [int(s) for s in check.split() if s.isdigit()]
            print("NUM SCRUB: ", str(num_scrub[0]), "\n")
            if num_scrub[0] > comparator: #if the number in check is greater than num_scrub then we don't want it
                with open(out_bad_bold_list, "a") as myfile: #making a file that lists all the bad ones
                    myfile.write("%s/%s\n"%(derivatives_dir, file))
                    print("wrote bad file")
                    myfile.close() 
    except FileNotFoundError:   
        print("FILE IS EMPTY, PASSING")

In [120]:
get_subjects()

>>>>>>----------->  sub-001
>>>>>>>>-----------------> STARTING MOTION CORRECTION ON  sub-001


UnboundLocalError: local variable 'plotz' referenced before assignment